# Comparison Recipes

In [187]:
import json
from icecream import ic

In [188]:
with open("../data/items.json") as f:
    items = json.load(f)

with open("../data/recipes.json") as f:
    recipes = json.load(f)

with open("../data/resources.json") as f:
    resources = json.load(f)

In [189]:
def find_recipes_with_product(item: str) -> dict:
    return {
        slug: recipe
        for slug, recipe in recipes.items()
        if any(products["item"] == item for products in recipe["products"])
    }

In [190]:
def find_recipes_for_product_with_no_byproducts(item: str) -> list:
    return {
        slug: recipe
        for slug, recipe in recipes.items()
        if len(recipe["products"]) == 1 and recipe["products"][0]["item"] == item
    }

In [191]:
def evaluate_recipes(item: str) -> list[tuple[int, str]]:
    recipe_for_item = find_recipes_for_product_with_no_byproducts(item)

    recipe_evalution = []
    for key, recipe in recipe_for_item.items():
        total_cost = 0
        product_amount = recipe["products"][0]["amount"]
        for ingredient in recipe["ingredients"]:
            item_name = ingredient["item"]
            if cost_list.get(item_name) is None:
                ic(item_name)
                raise NotImplementedError()
            total_cost += round(
                ingredient["amount"] * cost_list[item_name][1] / product_amount, 4
            )
        recipe_evalution.append([key, total_cost])
    recipe_evalution.sort(key=lambda x: x[1])
    return recipe_evalution


In [192]:
def evaluate_recipes_for_ore(ore: str) -> list[tuple[int, str]]:
    ore_recipes = evaluate_recipes(ore)
    ore_recipes.append(["extractor", resources[ore]["rarity"]])
    ore_recipes.sort(key=lambda x: x[1])
    return ore_recipes

In [193]:
cost_list = {
    key: ["extractor", resource["rarity"]] for key, resource in resources.items()
}

In [194]:
cost_list["sulfuric-acid"] = ["sulfuric_acid", resources["sulfur"]["rarity"]]
# ? sam to reanimated sam is 4:1
cost_list["reanimated-sam"] = ["reanimated-sam", resources["sam"]["rarity"] * 4]

In [195]:
iron_ingot = "iron-ingot"
iron_ingot_recipes = evaluate_recipes("iron-ingot")
cost_list[iron_ingot] = iron_ingot_recipes[0]
iron_ingot_recipes

[['recipe-alternate-pureironingot', 0.5385],
 ['recipe-alternate-ingotiron', 0.8666],
 ['recipe-ingotiron', 1.0],
 ['recipe-alternate-ironingot-leached', 1.353],
 ['recipe-alternate-ironingot-basic', 1.556]]

In [196]:
ore = "copper-ore"
ore_recipes = evaluate_recipes_for_ore(ore)
cost_list[ore] = ore_recipes[0]

In [197]:
ore = "bauxite"
ore_recipes = evaluate_recipes_for_ore(ore)
cost_list[ore] = ore_recipes[0]

In [198]:
ore = "caterium-ore"
ore_recipes = evaluate_recipes_for_ore(ore)
cost_list[ore] = ore_recipes[0]

In [201]:
ore = "raw-quartz"
ore_recipes = evaluate_recipes_for_ore(ore)
ic(ore_recipes)
cost_list[ore] = ore_recipes[0]

ic| ore_recipes: [['extractor', 6.82],
                  ['recipe-quartz-coal', 7.37],
                  ['recipe-quartz-bauxite', 8.6433]]


In [202]:
ore = "coal"
ore_recipes = evaluate_recipes_for_ore(ore)
ic(ore_recipes)
cost_list[ore] = ore_recipes[0]

ic| ore_recipes: [['extractor', 2.18],
                  ['recipe-coal-iron', 4.51],
                  ['recipe-coal-limestone', 6.97]]


In [200]:
cost_list

{'bauxite': ['recipe-bauxite-copper', 6.76],
 'caterium-ore': ['recipe-caterium-copper', 6.135],
 'coal': ['extractor', 2.18],
 'copper-ore': ['extractor', 2.5],
 'crude-oil': ['extractor', 7.31],
 'iron-ore': ['extractor', 1.0],
 'limestone': ['extractor', 1.32],
 'nitrogen-gas': ['extractor', 7.68],
 'raw-quartz': ['extractor', 6.82],
 'sulfur': ['extractor', 8.53],
 'uranium': ['extractor', 43.86],
 'sam': ['extractor', 9.03],
 'water': ['extractor', 0],
 'sulfuric-acid': ['sulfuric_acid', 8.53],
 'reanimated-sam': ['reanimated-sam', 36.12],
 'iron-ingot': ['recipe-alternate-pureironingot', 0.5385]}